In [1]:
!python -V

# Install the required package if not already installed
!pip install pyarrow

Python 3.10.13


In [2]:
import pandas as pd


url_january = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
df_january = pd.read_parquet(url_january)

num_columns_january = len(df_january.columns)
print(num_columns_january)

19


In [3]:
df_january['tpep_pickup_datetime'] = pd.to_datetime(df_january['tpep_pickup_datetime'])
df_january['tpep_dropoff_datetime'] = pd.to_datetime(df_january['tpep_dropoff_datetime'])
df_january['duration'] = (df_january['tpep_dropoff_datetime'] - df_january['tpep_pickup_datetime']).dt.total_seconds() / 60

std_duration = df_january['duration'].std()
print(std_duration)

42.59435124195458


In [4]:
filtered_df = df_january[(df_january['duration'] >= 1) & (df_january['duration'] <= 60)]

# Calculate the fraction of records left
fraction_left = len(filtered_df) / len(df_january)
print(fraction_left)

0.9812202822125979


In [5]:
from sklearn.feature_extraction import DictVectorizer

filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

data_dict = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
vectorizer = DictVectorizer(sparse=False)
feature_matrix = vectorizer.fit_transform(data_dict)
num_features = feature_matrix.shape[1]
print("Dimensionality of the feature matrix:", num_features)

/tmp/ipykernel_16535/580962251.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
/tmp/ipykernel_16535/580962251.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


MemoryError: Unable to allocate 11.5 GiB for an array with shape (3009173, 515) and data type float64

In [6]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import numpy as np

url_january = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
df_january = pd.read_parquet(url_january)

url_february = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
df_february = pd.read_parquet(url_february)

def preprocess_data(df):
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)
    return df

df_january = preprocess_data(df_january)
df_february = preprocess_data(df_february)

records_january = df_january[['PULocationID', 'DOLocationID']].to_dict(orient='records')
records_february = df_february[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(records_january)
X_val = dv.transform(records_february)

X_train = csr_matrix(X_train)
X_val = csr_matrix(X_val)

y_train = df_january['duration'].values
y_val = df_february['duration'].values

model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
print("Train RMSE:", rmse_train)

y_val_pred = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
print("Validation RMSE:", rmse_val)


: 